## Trial json file

In [27]:
import json
import pandas as pd

In [8]:
# Python program to read
# json file
 
# Opening JSON file
f = open('hyper_param.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
 
print(data)

{'Data': {'Sampling_size': 1980, 'task': {'Supervised': {'classes': {'4_classes': [4, 5, 8, 10], '10_classes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, 'tr_sz': 0.6, 'ts_sz': 0.4, 'balanced': 'True', 'min_sz': 70}}, 'Input_file': './Input_data/PCA_METABRIC_10cmp_IC10_full.csv', 'Output_dir': './Kernels_Results/'}, 'ft_maps': {'ft_map': ['ZZ', 'Z'], 'ent_type': ['linear', 'full'], 'n_reps': [1]}, 'Backend': {'shots': 2000, 'n_qubits': 4, 'backend': 'qasm_simulator', 'seed': 12345}, 'Scaling': {'bandwidth': [0.125, 0.25, 0.5, 1, 2]}}


In [22]:
task=list(data['Data']['task'].keys())
print(task[0])

Supervised


In [24]:
b=data['Data']['task'][task[0]]['balanced']


In [25]:
if b:
    print('wew')

wew


In [30]:
data_input = pd.read_csv("./Input_data/PCA_METABRIC_10cmp_IC10_full.csv", sep = ",")

In [32]:
data_dict={}
for case,class_ in data['Data']["task"][task[0]]['classes'].items():
    df_tot_sel=df_tot_sel=data_input.loc[data_input.IntClustMemb.isin(class_)]
    data_dict[case]=df_tot_sel


In [33]:
data_dict.keys()

dict_keys(['4_classes', '10_classes'])